In [3]:
! pip install tensorflow

In [4]:
import pandas as pd
#import gspread
#from gspread_dataframe import get_as_dataframe, set_with_dataframe
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D
from keras import regularizers, optimizers

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [2]:
train =pd.read_csv('data/training.csv')
test = pd.read_csv('data/test.csv')

In [3]:
import os
import zipfile
import io
from PIL import Image
import tqdm

In [4]:
from keras_preprocessing.image import ImageDataGenerator


In [42]:
datagen=ImageDataGenerator(rescale=1./255)
train_generator=datagen.flow_from_dataframe(dataframe=train, directory="data/images",
                                            x_col="image_name", y_col=["x1","x2","y1","y2"], class_mode= "other",
                                            target_size=(640,480), batch_size=32, shuffle=False)
y_train = train.drop("image_name", axis=1)

Found 14000 images.


In [40]:
print(train_generator.next())

(array([[[[0.0627451 , 0.0627451 , 0.0627451 ],
         [0.06666667, 0.06666667, 0.06666667],
         [0.06666667, 0.06666667, 0.06666667],
         ...,
         [0.07450981, 0.07450981, 0.07450981],
         [0.07450981, 0.07450981, 0.07450981],
         [0.07450981, 0.07450981, 0.07450981]],

        [[0.0627451 , 0.0627451 , 0.0627451 ],
         [0.0627451 , 0.0627451 , 0.0627451 ],
         [0.06666667, 0.06666667, 0.06666667],
         ...,
         [0.07450981, 0.07450981, 0.07450981],
         [0.07450981, 0.07450981, 0.07450981],
         [0.07450981, 0.07450981, 0.07450981]],

        [[0.0627451 , 0.0627451 , 0.0627451 ],
         [0.0627451 , 0.0627451 , 0.0627451 ],
         [0.06666667, 0.06666667, 0.06666667],
         ...,
         [0.07450981, 0.07450981, 0.07450981],
         [0.07450981, 0.07450981, 0.07450981],
         [0.07450981, 0.07450981, 0.07450981]],

        ...,

        [[0.35686275, 0.3921569 , 0.31764707],
         [0.3529412 , 0.38823533, 0.3137255 

In [29]:
model = Sequential()

In [30]:
model.add(Conv2D(32,(4,4),padding = 'same',input_shape=(640,480,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [31]:
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))

In [32]:
model.compile(optimizers.rmsprop(lr=0.0001),
loss="mean_squared_error", metrics=["accuracy"])

In [33]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

In [34]:
train_count = 0
def generator(data, n):
    global train_count
    if train_count >= train_generator.n:
        train_count = 0
    while True:
        batch_labels = data[train_count:train_count+n]
        train_count += n
        print(train_count)
        return np.array(batch_labels)

def combined_generator(gen1, y_data):
    x = gen1.next()
    yield x, generator(y_data, len(x))

In [ ]:
model.fit_generator(generator=train_generator,#combined_generator(train_generator, y_train),
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=2)

Epoch 1/10


  177/14000 [..............................] - ETA: 72:01:44 - loss: 118583.8672 - acc: 0.0000e+ - ETA: 36:48:45 - loss: 114602.4961 - acc: 0.5000   - ETA: 24:57:57 - loss: 114684.9245 - acc: 0.66 - ETA: 19:02:37 - loss: 110355.0098 - acc: 0.75 - ETA: 15:30:12 - loss: 110486.9859 - acc: 0.80 - ETA: 13:08:05 - loss: 110591.8841 - acc: 0.83 - ETA: 11:26:21 - loss: 108834.2009 - acc: 0.85 - ETA: 10:09:52 - loss: 105428.9082 - acc: 0.87 - ETA: 9:10:18 - loss: 102024.3021 - acc: 0.8889 - ETA: 8:22:38 - loss: 103297.0641 - acc: 0.900 - ETA: 7:43:45 - loss: 99888.0973 - acc: 0.909 - ETA: 7:11:15 - loss: 97133.3633 - acc: 0.91 - ETA: 6:43:46 - loss: 94054.8951 - acc: 0.84 - ETA: 6:20:11 - loss: 91176.1557 - acc: 0.78 - ETA: 5:59:48 - loss: 87793.0031 - acc: 0.80 - ETA: 5:41:56 - loss: 83508.9244 - acc: 0.81 - ETA: 5:26:08 - loss: 79741.3788 - acc: 0.82 - ETA: 5:12:10 - loss: 77518.2323 - acc: 0.83 - ETA: 4:59:38 - loss: 74647.1847 - acc: 0.84 - ETA: 4:48:19 - loss: 72145.7883 - acc: 0.85 - ETA

  711/14000 [>.............................] - ETA: 1:19:24 - loss: 18516.6168 - acc: 0.84 - ETA: 1:19:23 - loss: 18524.5375 - acc: 0.84 - ETA: 1:19:21 - loss: 18539.7239 - acc: 0.84 - ETA: 1:19:20 - loss: 18538.5272 - acc: 0.84 - ETA: 1:19:19 - loss: 18542.2925 - acc: 0.84 - ETA: 1:19:18 - loss: 18516.0114 - acc: 0.84 - ETA: 1:19:17 - loss: 18489.7268 - acc: 0.84 - ETA: 1:19:15 - loss: 18471.0055 - acc: 0.84 - ETA: 1:19:14 - loss: 18475.8676 - acc: 0.84 - ETA: 1:19:13 - loss: 18486.1098 - acc: 0.84 - ETA: 1:19:12 - loss: 18506.3528 - acc: 0.84 - ETA: 1:19:11 - loss: 18511.1087 - acc: 0.84 - ETA: 1:19:09 - loss: 18507.3077 - acc: 0.84 - ETA: 1:19:08 - loss: 18481.2884 - acc: 0.84 - ETA: 1:19:07 - loss: 18496.0084 - acc: 0.84 - ETA: 1:19:06 - loss: 18501.6722 - acc: 0.84 - ETA: 1:19:05 - loss: 18503.5726 - acc: 0.84 - ETA: 1:19:04 - loss: 18495.2695 - acc: 0.84 - ETA: 1:19:03 - loss: 18506.9348 - acc: 0.84 - ETA: 1:19:01 - loss: 18541.0364 - acc: 0.84 - ETA: 1:19:00 - loss: 18536.5917 -

  889/14000 [>.............................] - ETA: 1:16:31 - loss: 17725.4814 - acc: 0.85 - ETA: 1:16:30 - loss: 17707.3092 - acc: 0.85 - ETA: 1:16:30 - loss: 17726.5817 - acc: 0.85 - ETA: 1:16:29 - loss: 17714.5910 - acc: 0.85 - ETA: 1:16:28 - loss: 17696.2528 - acc: 0.85 - ETA: 1:16:27 - loss: 17683.4205 - acc: 0.85 - ETA: 1:16:26 - loss: 17665.8440 - acc: 0.85 - ETA: 1:16:26 - loss: 17647.4464 - acc: 0.85 - ETA: 1:16:25 - loss: 17632.8399 - acc: 0.85 - ETA: 1:16:24 - loss: 17621.0950 - acc: 0.85 - ETA: 1:16:23 - loss: 17598.5494 - acc: 0.85 - ETA: 1:16:23 - loss: 17613.4489 - acc: 0.85 - ETA: 1:16:22 - loss: 17595.7081 - acc: 0.85 - ETA: 1:16:21 - loss: 17591.1729 - acc: 0.85 - ETA: 1:16:20 - loss: 17581.6146 - acc: 0.85 - ETA: 1:16:20 - loss: 17612.7642 - acc: 0.85 - ETA: 1:16:19 - loss: 17609.6209 - acc: 0.85 - ETA: 1:16:18 - loss: 17607.9177 - acc: 0.85 - ETA: 1:16:17 - loss: 17601.3840 - acc: 0.85 - ETA: 1:16:17 - loss: 17579.4883 - acc: 0.85 - ETA: 1:16:16 - loss: 17564.6229 -

 1067/14000 [=>............................] - ETA: 1:14:27 - loss: 16773.2905 - acc: 0.86 - ETA: 1:14:27 - loss: 16787.3192 - acc: 0.86 - ETA: 1:14:26 - loss: 16770.8371 - acc: 0.86 - ETA: 1:14:25 - loss: 16764.8764 - acc: 0.86 - ETA: 1:14:25 - loss: 16759.3298 - acc: 0.86 - ETA: 1:14:24 - loss: 16753.5828 - acc: 0.86 - ETA: 1:14:23 - loss: 16742.3000 - acc: 0.86 - ETA: 1:14:23 - loss: 16729.0122 - acc: 0.86 - ETA: 1:14:22 - loss: 16735.2894 - acc: 0.86 - ETA: 1:14:22 - loss: 16726.8662 - acc: 0.86 - ETA: 1:14:21 - loss: 16725.5484 - acc: 0.86 - ETA: 1:14:20 - loss: 16739.9364 - acc: 0.86 - ETA: 1:14:20 - loss: 16729.8468 - acc: 0.86 - ETA: 1:14:19 - loss: 16735.4119 - acc: 0.86 - ETA: 1:14:18 - loss: 16721.9899 - acc: 0.86 - ETA: 1:14:18 - loss: 16713.3685 - acc: 0.85 - ETA: 1:14:17 - loss: 16703.9984 - acc: 0.85 - ETA: 1:14:17 - loss: 16690.1920 - acc: 0.86 - ETA: 1:14:16 - loss: 16680.4256 - acc: 0.86 - ETA: 1:14:15 - loss: 16668.2035 - acc: 0.86 - ETA: 1:14:15 - loss: 16652.6296 -

 1245/14000 [=>............................] - ETA: 1:12:47 - loss: 16167.5424 - acc: 0.84 - ETA: 1:12:46 - loss: 16165.0181 - acc: 0.84 - ETA: 1:12:45 - loss: 16247.5134 - acc: 0.84 - ETA: 1:12:45 - loss: 16254.6911 - acc: 0.84 - ETA: 1:12:44 - loss: 16252.9427 - acc: 0.84 - ETA: 1:12:44 - loss: 16245.1291 - acc: 0.84 - ETA: 1:12:43 - loss: 16245.1526 - acc: 0.84 - ETA: 1:12:43 - loss: 16249.5615 - acc: 0.84 - ETA: 1:12:42 - loss: 16241.8096 - acc: 0.84 - ETA: 1:12:42 - loss: 16241.5201 - acc: 0.84 - ETA: 1:12:42 - loss: 16231.7619 - acc: 0.84 - ETA: 1:12:41 - loss: 16251.9496 - acc: 0.84 - ETA: 1:12:40 - loss: 16255.5353 - acc: 0.84 - ETA: 1:12:40 - loss: 16247.2659 - acc: 0.84 - ETA: 1:12:39 - loss: 16245.4610 - acc: 0.84 - ETA: 1:12:39 - loss: 16232.7706 - acc: 0.84 - ETA: 1:12:38 - loss: 16220.8828 - acc: 0.84 - ETA: 1:12:38 - loss: 16212.3939 - acc: 0.84 - ETA: 1:12:37 - loss: 16211.5021 - acc: 0.84 - ETA: 1:12:37 - loss: 16218.9603 - acc: 0.84 - ETA: 1:12:36 - loss: 16224.0138 -

 1423/14000 [==>...........................] - ETA: 1:11:18 - loss: 15783.4352 - acc: 0.84 - ETA: 1:11:18 - loss: 15783.8060 - acc: 0.85 - ETA: 1:11:17 - loss: 15775.8390 - acc: 0.85 - ETA: 1:11:17 - loss: 15765.8689 - acc: 0.85 - ETA: 1:11:16 - loss: 15760.6927 - acc: 0.85 - ETA: 1:11:16 - loss: 15787.0027 - acc: 0.85 - ETA: 1:11:15 - loss: 15785.4517 - acc: 0.85 - ETA: 1:11:15 - loss: 15808.3239 - acc: 0.85 - ETA: 1:11:14 - loss: 15801.4507 - acc: 0.85 - ETA: 1:11:14 - loss: 15807.0482 - acc: 0.85 - ETA: 1:11:13 - loss: 15799.3859 - acc: 0.85 - ETA: 1:11:13 - loss: 15793.1554 - acc: 0.85 - ETA: 1:11:12 - loss: 15783.3355 - acc: 0.85 - ETA: 1:11:12 - loss: 15781.6521 - acc: 0.85 - ETA: 1:11:11 - loss: 15783.3092 - acc: 0.85 - ETA: 1:11:11 - loss: 15778.7434 - acc: 0.85 - ETA: 1:11:10 - loss: 15770.0599 - acc: 0.85 - ETA: 1:11:10 - loss: 15769.0351 - acc: 0.85 - ETA: 1:11:09 - loss: 15763.6806 - acc: 0.85 - ETA: 1:11:09 - loss: 15760.2029 - acc: 0.85 - ETA: 1:11:08 - loss: 15761.1968 -

 1601/14000 [==>...........................] - ETA: 1:09:55 - loss: 15479.2843 - acc: 0.85 - ETA: 1:09:55 - loss: 15476.7698 - acc: 0.85 - ETA: 1:09:54 - loss: 15466.8974 - acc: 0.85 - ETA: 1:09:54 - loss: 15466.3231 - acc: 0.85 - ETA: 1:09:53 - loss: 15466.5914 - acc: 0.85 - ETA: 1:09:53 - loss: 15465.4302 - acc: 0.85 - ETA: 1:09:53 - loss: 15461.1250 - acc: 0.85 - ETA: 1:09:52 - loss: 15458.8415 - acc: 0.85 - ETA: 1:09:52 - loss: 15451.1887 - acc: 0.85 - ETA: 1:09:51 - loss: 15447.1988 - acc: 0.85 - ETA: 1:09:51 - loss: 15439.5357 - acc: 0.85 - ETA: 1:09:50 - loss: 15432.4444 - acc: 0.85 - ETA: 1:09:50 - loss: 15426.9639 - acc: 0.85 - ETA: 1:09:50 - loss: 15430.3685 - acc: 0.85 - ETA: 1:09:49 - loss: 15423.6202 - acc: 0.85 - ETA: 1:09:49 - loss: 15425.4671 - acc: 0.85 - ETA: 1:09:48 - loss: 15426.1968 - acc: 0.85 - ETA: 1:09:48 - loss: 15469.3890 - acc: 0.85 - ETA: 1:09:47 - loss: 15465.6439 - acc: 0.85 - ETA: 1:09:47 - loss: 15476.7543 - acc: 0.85 - ETA: 1:09:47 - loss: 15475.0984 -

 1779/14000 [==>...........................] - ETA: 1:08:38 - loss: 15155.9723 - acc: 0.85 - ETA: 1:08:38 - loss: 15148.3159 - acc: 0.85 - ETA: 1:08:37 - loss: 15148.8092 - acc: 0.85 - ETA: 1:08:37 - loss: 15151.5561 - acc: 0.85 - ETA: 1:08:36 - loss: 15147.1591 - acc: 0.85 - ETA: 1:08:36 - loss: 15142.7358 - acc: 0.85 - ETA: 1:08:36 - loss: 15136.9522 - acc: 0.85 - ETA: 1:08:35 - loss: 15137.7920 - acc: 0.85 - ETA: 1:08:35 - loss: 15143.4280 - acc: 0.85 - ETA: 1:08:34 - loss: 15136.8931 - acc: 0.85 - ETA: 1:08:34 - loss: 15132.5842 - acc: 0.85 - ETA: 1:08:33 - loss: 15131.8925 - acc: 0.85 - ETA: 1:08:33 - loss: 15130.2391 - acc: 0.85 - ETA: 1:08:33 - loss: 15127.2291 - acc: 0.85 - ETA: 1:08:32 - loss: 15125.2704 - acc: 0.85 - ETA: 1:08:32 - loss: 15122.9208 - acc: 0.85 - ETA: 1:08:31 - loss: 15117.3072 - acc: 0.85 - ETA: 1:08:31 - loss: 15117.9247 - acc: 0.85 - ETA: 1:08:31 - loss: 15120.6688 - acc: 0.85 - ETA: 1:08:30 - loss: 15116.2063 - acc: 0.85 - ETA: 1:08:30 - loss: 15120.0452 -

 1895/14000 [===>..........................] - ETA: 1:07:25 - loss: 14917.8423 - acc: 0.85 - ETA: 1:07:25 - loss: 14914.0034 - acc: 0.85 - ETA: 1:07:24 - loss: 14909.2419 - acc: 0.85 - ETA: 1:07:24 - loss: 14904.4730 - acc: 0.85 - ETA: 1:07:24 - loss: 14902.1585 - acc: 0.85 - ETA: 1:07:23 - loss: 14900.9526 - acc: 0.85 - ETA: 1:07:23 - loss: 14899.6844 - acc: 0.85 - ETA: 1:07:22 - loss: 14899.1182 - acc: 0.85 - ETA: 1:07:22 - loss: 14900.5703 - acc: 0.85 - ETA: 1:07:22 - loss: 14902.6478 - acc: 0.85 - ETA: 1:07:21 - loss: 14895.4562 - acc: 0.85 - ETA: 1:07:21 - loss: 14889.3613 - acc: 0.85 - ETA: 1:07:20 - loss: 14886.1936 - acc: 0.85 - ETA: 1:07:20 - loss: 14880.3290 - acc: 0.85 - ETA: 1:07:20 - loss: 14886.2946 - acc: 0.85 - ETA: 1:07:19 - loss: 14890.7657 - acc: 0.85 - ETA: 1:07:19 - loss: 14893.3554 - acc: 0.85 - ETA: 1:07:19 - loss: 14887.3322 - acc: 0.85 - ETA: 1:07:18 - loss: 14881.3290 - acc: 0.85 - ETA: 1:07:18 - loss: 14882.2603 - acc: 0.85 - ETA: 1:07:18 - loss: 14886.1754 -